In [1]:
import warnings
warnings.filterwarnings('ignore')

In [45]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [137]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [48]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [107]:
#Lematizacion 
# Importación librerías
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

# Importación de librerias
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

# Definición de la función que tenga como parámetro texto y devuelva una lista de lemas
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

# Creación de matrices de documentos usando CountVectorizer, usando el parámetro 'split_into_lemmas'
vect_lemas = CountVectorizer(stop_words='english', analyzer=split_into_lemmas)

X_dtm = vect_lemas.fit_transform(dataTraining['plot'])
X_dtm.shape

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrador\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(7895, 34629)

In [93]:
# Definición de variables predictoras (X)
vect = CountVectorizer(stop_words='english')
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 38370)

In [138]:
#Bigramas 
# Definición de variables predictoras (X)
vect = CountVectorizer(stop_words='english', ngram_range=(1, 3))
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 924096)

In [144]:
# Idf 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
tfidf_transformer = TfidfTransformer()
vect = CountVectorizer(stop_words='english')
X_dtm_1 = vect.fit_transform(dataTraining['plot'])

X_dtm = tfidf_transformer.fit_transform(X_dtm_1)
X_dtm.shape 


(7895, 38370)

In [139]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [145]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [146]:
%%time
# Definición y entrenamiento
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=30, random_state=42))
clf.fit(X_train, y_train_genres)

Wall time: 1min 5s


OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=30, n_jobs=-1,
                                                     random_state=42))

In [147]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

# Sin lemas 0.7963860628253788
# COn lemas 0.7983106666028829
# COn bigramas 0.7737576554408515
# Con IDF 0.807300692490283

0.807300692490283

In [98]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf.predict_proba(X_test_dtm)

In [148]:
%%time
from sklearn.linear_model import LogisticRegression
# Definición y entrenamiento
lr = OneVsRestClassifier(LogisticRegression(random_state=123, solver='lbfgs', max_iter=3000, C=0.01, n_jobs=-1))
lr.fit(X_train, y_train_genres)

Wall time: 31.5 s


OneVsRestClassifier(estimator=LogisticRegression(C=0.01, max_iter=3000,
                                                 n_jobs=-1, random_state=123))

In [149]:
# Predicción del modelo de clasificación
y_pred_genres = lr.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

# Sin lematizacion 0.8591001418951785
# Con lematizacion 0.8361467846711504
# Sin lematizar con bigramas 0.8598116682546477
# sin Lematizar con trigramas 0.8590851191421395
# Con iDF 0.8710286526905197

0.8710286526905197

In [153]:
%%time

# Definición y entrenamiento
lr2 = OneVsRestClassifier(LogisticRegression(random_state=123, n_jobs=-1))
lr2.fit(X_train, y_train_genres)

# Predicción del modelo de clasificación
y_pred_genres = lr2.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

Wall time: 45 s


0.875797380865353

In [156]:
%%time
from sklearn.tree import DecisionTreeClassifier

dt = OneVsRestClassifier(DecisionTreeClassifier(random_state=123))
dt.fit(X_train, y_train_genres)
y_pred = dt.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred, average='macro')

Wall time: 3min 38s


0.6087586038996261

In [158]:
%%time
from sklearn.svm import SVC

svm = OneVsRestClassifier(SVC(random_state=123))
svm.fit(X_train, y_train_genres)
y_pred = svm.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred, average='macro')

AttributeError: This 'OneVsRestClassifier' has no attribute 'predict_proba'

In [159]:
%%time
from xgboost import XGBClassifier

xgb = OneVsRestClassifier(XGBClassifier(random_state=123))
xgb.fit(X_train, y_train_genres)
y_pred = xgb.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred, average='macro')

Wall time: 1min 29s


0.8180184843496011

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier

adaboost = OneVsRestClassifier(AdaBoostClassifier(random_state=123))
adaboost.fit(X_train, y_train_genres)
y_pred = adaboost.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred, average='macro')

In [150]:
# transformación variables predictoras X del conjunto de test
X_test_dtm_1 = vect.transform(dataTesting['plot'])
X_test_dtm = tfidf_transformer.transform(X_test_dtm_1)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = lr.predict_proba(X_test_dtm)

In [151]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_LR_TIDF.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.165744,0.129005,0.030439,0.045920,0.381150,0.183916,0.054736,0.507937,0.081135,0.088287,...,0.032988,0.097578,0.000756,0.243049,0.089385,0.011730,0.031531,0.256486,0.046371,0.028791
4,0.166500,0.128163,0.030459,0.046584,0.377521,0.187487,0.055130,0.513134,0.080701,0.086803,...,0.032932,0.096791,0.000756,0.234943,0.088403,0.011726,0.031613,0.257204,0.046794,0.028850
5,0.165753,0.127490,0.030296,0.046059,0.376974,0.191229,0.054563,0.513814,0.080398,0.086930,...,0.032796,0.100116,0.000756,0.236896,0.089379,0.011705,0.031499,0.263524,0.046350,0.028717
6,0.166104,0.128978,0.030314,0.045979,0.377979,0.183421,0.054776,0.511266,0.080968,0.087274,...,0.032923,0.097965,0.000756,0.238365,0.089955,0.011686,0.031700,0.259563,0.046854,0.028664
7,0.165001,0.128623,0.030408,0.045704,0.376600,0.184472,0.054639,0.503293,0.081034,0.088423,...,0.032853,0.099003,0.000756,0.236823,0.092036,0.011716,0.031392,0.261388,0.046294,0.028688


Ejemplo interesante sobre algo similar..

https://github.com/tomkeith/Multi-label-classification-with-NLP


Ejemplo de análisis 

https://www.kaggle.com/code/saisandeepjallepalli/imdb-5000-eda-and-data-visualization